In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
# import unidecode
# import contractions
import re
# from word2number import w2n
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Keras package
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, LSTM, Lambda, Bidirectional
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.pooling import GlobalAveragePooling1D
import keras.backend as K

In [2]:
## initialize global variables

MAX_SEQUENCE_LENGTH = 60  
MAX_NUM_WORDS = 200000  # There are about 201000 unique words in training dataset, 200000 is enough for tokenization
EMBEDDING_DIM = 300  # word-embedded-vector dimension(300 is for 'glove.42B.300d')
N_HIDDEN = 512
N_DENSE = 256

DROPOUT_RATE_LSTM = 0.30 # drop-out possibility, random set to avoid outfitting
DROPOUT_RATE_DENSE = 0.30

ACTIVE_FUNC = 'relu'
VERSION = 'bilstm'

PATH_TO_GLOVE_FILE = './data/glove.42B.300d.txt'

In [3]:
print(f'Nodes Hidden: {N_HIDDEN}')
print(f'Nodes Dense: {N_DENSE}')

print(f'Dropout Rate LSTM: {DROPOUT_RATE_LSTM}')
print(f'Dropout Rate Dense: {DROPOUT_RATE_DENSE}')


Nodes Hidden: 512
Nodes Dense: 256
Dropout Rate LSTM: 0.3
Dropout Rate Dense: 0.3


In [4]:
# Create word embedding dictionary from 'glove.840B.300d.txt', {key:value} is {word: glove vector(300,)}
print('Create word embedding dictionary')

embeddings_index = {}
with open(PATH_TO_GLOVE_FILE) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs
        
print("Found %s word vectors." % len(embeddings_index))

Create word embedding dictionary
Found 1917494 word vectors.


In [5]:
df = pd.read_csv('./data/questions.csv.zip')
df.head()
# df.tail()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
print(df[df['is_duplicate'] == 1].head())
print(f'\ncount: \n{df.count()}')
print(f"\nsum is_duplicate: {df['is_duplicate'].sum()}")

    id  qid1  qid2                                          question1  \
5    5    11    12  Astrology: I am a Capricorn Sun Cap moon and c...   
7    7    15    16                     How can I be a good geologist?   
11  11    23    24        How do I read and find my YouTube comments?   
12  12    25    26               What can make Physics easy to learn?   
13  13    27    28        What was your first sexual experience like?   

                                            question2  is_duplicate  
5   I'm a triple Capricorn (Sun, Moon and ascendan...             1  
7           What should I do to be a great geologist?             1  
11             How can I see all my Youtube comments?             1  
12            How can you make physics easy to learn?             1  
13             What was your first sexual experience?             1  

count: 
id              404351
qid1            404351
qid2            404351
question1       404350
question2       404349
is_duplicate    4

In [7]:
from sklearn.model_selection import train_test_split
y = df['is_duplicate']
# dft = df.drop('is_duplicate', axis=1)
dft = df
X_train, X_test, y_train, y_test = train_test_split(dft, y, test_size=0.95, random_state=42)



In [8]:
print(f"\nsum is_duplicate: {X_train['is_duplicate'].sum()}")
X_train.count()


sum is_duplicate: 7521


id              20217
qid1            20217
qid2            20217
question1       20217
question2       20217
is_duplicate    20217
dtype: int64

In [9]:
X_test.count()

id              384134
qid1            384134
qid2            384134
question1       384133
question2       384132
is_duplicate    384134
dtype: int64

In [10]:

train_q1 = X_train['question1'].values
train_q2 = X_train['question2'].values
train_labels = X_train['is_duplicate'].values

sample = X_test.sample(n=10000, random_state=42)
test_q1 = X_test['question1'].values
test_q2 =X_test['question1'].values
test_labels = X_test['is_duplicate'].values
test_ids = X_test['id'].values  # id..

In [11]:
# Preprocess text in dataset
print('Processing text dataset')

def text_to_wordlist(text):
    
    # split words
    text = str(text).split()
    
    text = " ".join(text)

    # Use re to clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text, re.IGNORECASE)
    text = re.sub(r"what's", "what is ", text, re.IGNORECASE)
    text = re.sub(r"\’s", " ", text, re.IGNORECASE)
    text = re.sub(r"\'s", " ", text, re.IGNORECASE)
    text = re.sub(r"\'ve", " have ", text, re.IGNORECASE)
    text = re.sub(r"can't", "cannot ", text, re.IGNORECASE)
    text = re.sub(r"n't", " not ", text, re.IGNORECASE)
    text = re.sub(r"i'm", "i am ", text, re.IGNORECASE)
    text = re.sub(r"\'re", " are ", text, re.IGNORECASE)
    text = re.sub(r"\'d", " would ", text, re.IGNORECASE)
    text = re.sub(r"\'ll", " will ", text, re.IGNORECASE)
    text = re.sub(r"\‘", " ", text, re.IGNORECASE)
    text = re.sub(r"\’", " ", text, re.IGNORECASE)
    text = re.sub(r"\"", " ", text, re.IGNORECASE)
    text = re.sub(r"\“", " ", text, re.IGNORECASE)
    text = re.sub(r"\”", " ", text, re.IGNORECASE)
    text = re.sub(r",", " ", text, re.IGNORECASE)
    text = re.sub(r"\.", " ", text, re.IGNORECASE)
    text = re.sub(r"!", " ! ", text, re.IGNORECASE)
    text = re.sub(r"\/", " ", text, re.IGNORECASE)
    text = re.sub(r"\^", " ^ ", text, re.IGNORECASE)
    text = re.sub(r"\+", " + ", text, re.IGNORECASE)
    text = re.sub(r"\-", " - ", text, re.IGNORECASE)
    text = re.sub(r"\=", " = ", text, re.IGNORECASE)
    text = re.sub(r"'", " ", text, re.IGNORECASE)
    text = re.sub(r":", " : ", text, re.IGNORECASE)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text, re.IGNORECASE)
    text = re.sub(r" e g ", " eg ", text, re.IGNORECASE)
    text = re.sub(r" b g ", " bg ", text, re.IGNORECASE)
    text = re.sub(r" u s ", " american ", text, re.IGNORECASE)
    text = re.sub(r" 9 11 ", "911", text, re.IGNORECASE)
    text = re.sub(r"e - mail", "email", text, re.IGNORECASE)
    text = re.sub(r"j k", "jk", text, re.IGNORECASE)
    text = re.sub(r"\s{2,}", " ", text, re.IGNORECASE)
    text = re.sub(r"\？", " ", text, re.IGNORECASE)
    
    # Return a list of words
    return(text)

train_text_q1 = [] # preprocessed text of q1
train_text_q2 = [] # preprocessed text of q2

text_set = set() # complete set of words for building embeddings

for text in train_q1:
    tt = text_to_wordlist(text)
    text_set.add(tt)
    train_text_q1.append(tt)
for text in train_q2:
    tt = text_to_wordlist(text)
    text_set.add(tt)
    train_text_q2.append(tt)

test_text_q1 = [] # preprocessed text of q1
test_text_q2 = [] # preprocessed text of q2

for text in test_q1:
    tt = text_to_wordlist(text)
    text_set.add(tt)
    test_text_q1.append(tt)
for text in test_q2:
    tt = text_to_wordlist(text)
    text_set.add(tt)
    test_text_q2.append(tt)

train_test_text = list(text_set)

vectorizer = TextVectorization(max_tokens=MAX_NUM_WORDS, output_sequence_length=EMBEDDING_DIM)
text_ds = tf.data.Dataset.from_tensor_slices(train_test_text).batch(128)
vectorizer.adapt(text_ds)
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
print(f'word_index len: {len(word_index)}')

Processing text dataset


2021-09-19 21:16:05.842439: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 21:16:05.848164: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 21:16:05.849578: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-19 21:16:05.850867: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

word_index len: 71373


In [12]:
# Keras.Tokenize for all text:
# First construct a Tokenizer()
# Then use tokenizer_on_texts() method to learn the dictionary of the corpus(all texts(sentences)). 
#    We can use .word_index to map between the each word (distinct) with the corresponding number.
# Then use text_to_sequence() method to transfer every text(sentence) in texts into sequences of word_indexes.
# Then add the same length by padding method: padding_sequences().
# Finally use the embedding layer in keras to carry out a vectorization, and input it into LSTM.

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(train_text_q1 + train_text_q2 + test_text_q1 + test_text_q2)  # generate a token dictionary, 

train_sequences_1 = tokenizer.texts_to_sequences(train_text_q1)  # sequence of q1
train_sequences_2 = tokenizer.texts_to_sequences(train_text_q2)  # sequence of q2
test_sequences_1 = tokenizer.texts_to_sequences(test_text_q1)  # sequence of q1_test
test_sequences_2 = tokenizer.texts_to_sequences(test_text_q2)  # sequence of q2_test

word_index = tokenizer.word_index
print('{} unique tokens are found'.format(len(word_index)))

# Pad all train with Max_Sequence_Length: 60
train_data_1 = pad_sequences(train_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q1 as train_data
train_data_2 = pad_sequences(train_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q2 as train_data
print('Shape of train data tensor:', train_data_1.shape)
print('Shape of train labels tensor:', train_labels.shape)

# Pad all test with Max_Sequence_Length
test_data_1 = pad_sequences(test_sequences_1, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q1_test as test_data
test_data_2 = pad_sequences(test_sequences_2, maxlen=MAX_SEQUENCE_LENGTH)  # padded_sequence of q2_test as test_data
print('Shape of test data vtensor:', test_data_2.shape)
print('Shape of test ids tensor:', test_ids.shape)

69955 unique tokens are found
Shape of train data tensor: (20217, 60)
Shape of train labels tensor: (20217,)
Shape of test data vtensor: (384134, 60)
Shape of test ids tensor: (384134,)


In [13]:
num_tokens = len(voc) + 2
hits = 0
misses = 0
misses_txt = []

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
        misses_txt.append(word)
print("Converted %d words (%d misses)" % (hits, misses))
# print(f'misses: {misses_txt}')

embedding_layer = Embedding(
    num_tokens,
    EMBEDDING_DIM,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

# BiLSTM layer
from keras.layers import Bidirectional, LSTM
lstm_layer = Bidirectional(LSTM(N_HIDDEN, dropout=DROPOUT_RATE_LSTM, recurrent_dropout=DROPOUT_RATE_LSTM))


# Define inputs
seq1 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
seq2 = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

# Run inputs through embedding
emb1 = embedding_layer(seq1)
emb2 = embedding_layer(seq2)

# Run through LSTM layers
lstm_a = lstm_layer(emb1)
lstm_b = lstm_layer(emb2)

merged = concatenate([lstm_a, lstm_b])
merged = BatchNormalization()(merged)
merged = Dropout(DROPOUT_RATE_DENSE)(merged)

merged = Dense(N_DENSE, activation=ACTIVE_FUNC)(merged)
merged = BatchNormalization()(merged)
merged = Dropout(DROPOUT_RATE_DENSE)(merged)

preds = Dense(1, activation='sigmoid')(merged)

Converted 61353 words (8602 misses)


In [ ]:
# Train the model

print('Starting the model training')

model = Model(inputs=[seq1, seq2], outputs=preds)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

# Summerization of model
model.summary()

# Set early stopping (large patience should be useful)
early_stopping =EarlyStopping(monitor='val_loss', patience=10)
bst_model_path = VERSION + '.h5' 
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)


hist = model.fit([train_data_1, train_data_2], train_labels, \
        validation_split=0.02, \
        epochs=10, batch_size=128, shuffle=True, \
        callbacks=[early_stopping, model_checkpoint])

model.load_weights(bst_model_path) # sotre model parameters in .h5 file
bst_val_score = min(hist.history['val_loss'])




Starting the model training
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 60)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 60, 300)      21412500    input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1024)         3330048     e

In [ ]:
# print(f'embedding misses: {misses_txt}')